In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [35]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [36]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'yeong_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 200)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)
#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['지석천_2015.xlsx', '지석천_2016.xlsx', '지석천_2017.xlsx', '지석천_2018.xlsx', '지석천_2019.xlsx', '지석천_2020.xlsx'], ['용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/yeong/자동/지석천_2015.xlsx
data/yeong/자동/지석천_2016.xlsx
data/yeong/자동/지석천_2017.xlsx
data/yeong/자동/지석천_2018.xlsx
data/yeong/자동/지석천_2019.xlsx
data/yeong/자동/지석천_2020.xlsx
You must check this make_timeseries : time 

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7800 - disc_loss: 0.3614 - rmse: 0.3846 - val_loss: 0.4252
Epoch 48/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.4483 - disc_loss: 0.3555 - rmse: 0.3998 - val_loss: 0.4031
Epoch 49/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.1408 - disc_loss: 0.3544 - rmse: 0.4161 - val_loss: 0.4642
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.7486 - disc_loss: 0.3295 - rmse: 0.4486 - val_loss: 0.4322
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.1171 - disc_loss: 0.3283 - rmse: 0.3963 - val_loss: 0.4408
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.8748 - disc_loss: 0.3280 - rmse: 0.4296 - val_loss: 0.4279
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.7639 - disc_loss: 0.3487 - rmse: 0.4696 - val_loss: 0.4073
Epoch 54/2000
1/1 [================

Epoch 106/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5881 - disc_loss: 0.2774 - rmse: 0.3457 - val_loss: 0.4265
Epoch 107/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4965 - disc_loss: 0.2719 - rmse: 0.3448 - val_loss: 0.3595
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3793 - disc_loss: 0.2562 - rmse: 0.3501 - val_loss: 0.3847
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6274 - disc_loss: 0.2680 - rmse: 0.3540 - val_loss: 0.3423
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3022 - disc_loss: 0.2728 - rmse: 0.3350 - val_loss: 0.3339
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0638 - disc_loss: 0.2678 - rmse: 0.3576 - val_loss: 0.3295
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7194 - disc_loss: 0.2585 - rmse: 0.3145 - val_loss: 0.3650
Epoch 113/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3226 - disc_loss: 0.2497 - rmse: 0.2783 - val_loss: 0.3219
Epoch 165/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8260 - disc_loss: 0.2585 - rmse: 0.3612 - val_loss: 0.3537
Epoch 166/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.8151 - disc_loss: 0.2534 - rmse: 0.3448 - val_loss: 0.3110
Epoch 167/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.6998 - disc_loss: 0.2406 - rmse: 0.3383 - val_loss: 0.3009
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1018 - disc_loss: 0.2395 - rmse: 0.2969 - val_loss: 0.2812
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0453 - disc_loss: 0.2557 - rmse: 0.3042 - val_loss: 0.3535
Epoch 170/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.9077 - disc_loss: 0.2345 - rmse: 0.3087 - val_loss: 0.3639
Epoch 171/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9922 - disc_loss: 0.2196 - rmse: 0.3672 - val_loss: 0.3162
Epoch 223/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5166 - disc_loss: 0.2269 - rmse: 0.3760 - val_loss: 0.3326
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2830 - disc_loss: 0.2235 - rmse: 0.3890 - val_loss: 0.2967
Epoch 225/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.6931 - disc_loss: 0.2470 - rmse: 0.3570 - val_loss: 0.3251
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3272 - disc_loss: 0.2243 - rmse: 0.3208 - val_loss: 0.3209
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6847 - disc_loss: 0.2499 - rmse: 0.3538 - val_loss: 0.3103
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9745 - disc_loss: 0.2406 - rmse: 0.3560 - val_loss: 0.3780
Epoch 229/2000
1/1 [===============

...

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5338 - disc_loss: 0.1513 - rmse: 0.2781 - val_loss: 0.2377
Epoch 1731/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2752 - disc_loss: 0.1443 - rmse: 0.2310 - val_loss: 0.3043
Epoch 1732/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.2211 - disc_loss: 0.1645 - rmse: 0.2854 - val_loss: 0.2352
Epoch 1733/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.8894 - disc_loss: 0.1505 - rmse: 0.2089 - val_loss: 0.3072
Epoch 1734/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8296 - disc_loss: 0.1629 - rmse: 0.2102 - val_loss: 0.3095
Epoch 1735/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.7223 - disc_loss: 0.1509 - rmse: 0.2527 - val_loss: 0.2446
Epoch 1736/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4865 - disc_loss: 0.1528 - rmse: 0.2724 - val_loss: 0.2493
Epoch 1737/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4387 - disc_loss: 0.1450 - rmse: 0.2833 - val_loss: 0.2471
Epoch 1789/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8071 - disc_loss: 0.1696 - rmse: 0.2785 - val_loss: 0.2516
Epoch 1790/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.9584 - disc_loss: 0.1516 - rmse: 0.2989 - val_loss: 0.2236
Epoch 1791/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1389 - disc_loss: 0.1502 - rmse: 0.2530 - val_loss: 0.2264
Epoch 1792/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1164 - disc_loss: 0.1509 - rmse: 0.2425 - val_loss: 0.2360
Epoch 1793/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8829 - disc_loss: 0.1523 - rmse: 0.2807 - val_loss: 0.2122
Epoch 1794/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0938 - disc_loss: 0.1575 - rmse: 0.2216 - val_loss: 0.2589
Epoch 1795/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6409 - disc_loss: 0.1504 - rmse: 0.2811 - val_loss: 0.2779
Epoch 1847/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5848 - disc_loss: 0.1547 - rmse: 0.2971 - val_loss: 0.2764
Epoch 1848/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1146 - disc_loss: 0.1491 - rmse: 0.2677 - val_loss: 0.2349
Epoch 1849/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9981 - disc_loss: 0.1560 - rmse: 0.3022 - val_loss: 0.2786
Epoch 1850/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9780 - disc_loss: 0.1619 - rmse: 0.2460 - val_loss: 0.2656
Epoch 1851/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5153 - disc_loss: 0.1623 - rmse: 0.2645 - val_loss: 0.3013
Epoch 1852/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.0963 - disc_loss: 0.1619 - rmse: 0.2873 - val_loss: 0.2538
Epoch 1853/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8105 - disc_loss: 0.1446 - rmse: 0.2425 - val_loss: 0.2431
Epoch 1905/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3932 - disc_loss: 0.1493 - rmse: 0.2206 - val_loss: 0.2675
Epoch 1906/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0695 - disc_loss: 0.1500 - rmse: 0.3410 - val_loss: 0.2406
Epoch 1907/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1539 - disc_loss: 0.1482 - rmse: 0.2231 - val_loss: 0.3151
Epoch 1908/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8678 - disc_loss: 0.1517 - rmse: 0.2498 - val_loss: 0.3006
Epoch 1909/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0755 - disc_loss: 0.1535 - rmse: 0.2325 - val_loss: 0.2544
Epoch 1910/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3531 - disc_loss: 0.1543 - rmse: 0.2697 - val_loss: 0.2587
Epoch 1911/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0885 - disc_loss: 0.1539 - rmse: 0.2845 - val_loss: 0.3026
Epoch 1963/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8498 - disc_loss: 0.1615 - rmse: 0.2506 - val_loss: 0.2950
Epoch 1964/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1527 - disc_loss: 0.1553 - rmse: 0.2411 - val_loss: 0.2413
Epoch 1965/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7322 - disc_loss: 0.1524 - rmse: 0.3133 - val_loss: 0.2585
Epoch 1966/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3000 - disc_loss: 0.1595 - rmse: 0.3104 - val_loss: 0.2729
Epoch 1967/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3659 - disc_loss: 0.1477 - rmse: 0.2725 - val_loss: 0.2890
Epoch 1968/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5968 - disc_loss: 0.1465 - rmse: 0.2735 - val_loss: 0.2648
MissData :  save/  miss :  (1

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#
#
#
#
#
#
#-----------------------------------------------------------------------------------------------------------------
#
#
#
#
#
#
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [16]:
print(real_df_all.shape)

rnn_target_column = 'do'
print(rnn_target_column)

(52608, 48)
do


In [68]:
#train_df, val_df, test_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
teg_check = 'test=1, train=8, val=1'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (42087, 48) val_df.shape :  (5261, 48) test_df.shape: (5260, 48)


In [69]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
plt.figure()
plt.plot(test_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [72]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  2
out_num_features :  1


In [73]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [74]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/do/


## 모델 학습

In [ ]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/200
1/1 [==============================] - ETA: 0s - loss: 1.0411 - mean_absolute_error: 0.8335 - nse: -0.0106
Epoch 00001: val_loss improved from inf to 0.37577, saving model to save/best_model.h5
1/1 [==============================] - 4s 4s/step - loss: 1.0411 - mean_absolute_error: 0.8335 - nse: -0.0106 - val_loss: 0.3758 - val_mean_absolute_error: 0.4502 - val_nse: -0.0443
Epoch 2/200
1/1 [==============================] - ETA: 0s - loss: 1.0774 - mean_absolute_error: 0.8606 - nse: 0.0182
Epoch 00002: val_loss improved from 0.37577 to 0.35418, saving model to save/best_model.h5
1/1 [==============================] - 3s 3s/step - loss: 1.0774 - mean_absolute_error: 0.8606 - nse: 0.0182 - val_loss: 0.3542 - val_mean_absolute_error: 0.4325 - val_nse: 0.0109
Epoch 3/200
1/1 [==============================] - ETA: 0s - loss: 0.9150 - mean_absolute_error: 0.7686 - nse: 0.0770
Epoch 00003: val_loss improved from 0.35418 to 0.32476, saving model to save/best_model.h5
1/1 [=========

Epoch 23/200
1/1 [==============================] - ETA: 0s - loss: 0.3680 - mean_absolute_error: 0.4631 - nse: 0.6436
Epoch 00023: val_loss improved from 0.27485 to 0.27170, saving model to save/best_model.h5
1/1 [==============================] - 3s 3s/step - loss: 0.3680 - mean_absolute_error: 0.4631 - nse: 0.6436 - val_loss: 0.2717 - val_mean_absolute_error: 0.4040 - val_nse: 0.2330
Epoch 24/200
1/1 [==============================] - ETA: 0s - loss: 0.3582 - mean_absolute_error: 0.4471 - nse: 0.6102
Epoch 00024: val_loss improved from 0.27170 to 0.26125, saving model to save/best_model.h5
1/1 [==============================] - 3s 3s/step - loss: 0.3582 - mean_absolute_error: 0.4471 - nse: 0.6102 - val_loss: 0.2613 - val_mean_absolute_error: 0.3979 - val_nse: 0.2619
Epoch 25/200
1/1 [==============================] - ETA: 0s - loss: 0.3152 - mean_absolute_error: 0.4289 - nse: 0.7262
Epoch 00025: val_loss did not improve from 0.26125
1/1 [==============================] - 3s 3s/step 

1/1 [==============================] - ETA: 0s - loss: 0.3314 - mean_absolute_error: 0.4321 - nse: 0.6886
Epoch 00045: val_loss did not improve from 0.22225
1/1 [==============================] - 3s 3s/step - loss: 0.3314 - mean_absolute_error: 0.4321 - nse: 0.6886 - val_loss: 0.2267 - val_mean_absolute_error: 0.3574 - val_nse: 0.3653
Epoch 46/200
1/1 [==============================] - ETA: 0s - loss: 0.3830 - mean_absolute_error: 0.4537 - nse: 0.5641
Epoch 00046: val_loss did not improve from 0.22225
1/1 [==============================] - 3s 3s/step - loss: 0.3830 - mean_absolute_error: 0.4537 - nse: 0.5641 - val_loss: 0.2350 - val_mean_absolute_error: 0.3596 - val_nse: 0.3428
Epoch 47/200
1/1 [==============================] - ETA: 0s - loss: 0.3103 - mean_absolute_error: 0.4246 - nse: 0.6752
Epoch 00047: val_loss did not improve from 0.22225
1/1 [==============================] - 3s 3s/step - loss: 0.3103 - mean_absolute_error: 0.4246 - nse: 0.6752 - val_loss: 0.2487 - val_mean_abso

1/1 [==============================] - ETA: 0s - loss: 0.3512 - mean_absolute_error: 0.4482 - nse: 0.6584
Epoch 00068: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3512 - mean_absolute_error: 0.4482 - nse: 0.6584 - val_loss: 0.2165 - val_mean_absolute_error: 0.3495 - val_nse: 0.3908
Epoch 69/200
1/1 [==============================] - ETA: 0s - loss: 0.3335 - mean_absolute_error: 0.4172 - nse: 0.6311
Epoch 00069: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3335 - mean_absolute_error: 0.4172 - nse: 0.6311 - val_loss: 0.2188 - val_mean_absolute_error: 0.3545 - val_nse: 0.3863
Epoch 70/200
1/1 [==============================] - ETA: 0s - loss: 0.3017 - mean_absolute_error: 0.4053 - nse: 0.7201
Epoch 00070: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3017 - mean_absolute_error: 0.4053 - nse: 0.7201 - val_loss: 0.2189 - val_mean_abso

Epoch 92/200
1/1 [==============================] - ETA: 0s - loss: 0.3477 - mean_absolute_error: 0.4290 - nse: 0.6553
Epoch 00092: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3477 - mean_absolute_error: 0.4290 - nse: 0.6553 - val_loss: 0.2468 - val_mean_absolute_error: 0.3649 - val_nse: 0.3080
Epoch 93/200
1/1 [==============================] - ETA: 0s - loss: 0.2771 - mean_absolute_error: 0.3888 - nse: 0.6802
Epoch 00093: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.2771 - mean_absolute_error: 0.3888 - nse: 0.6802 - val_loss: 0.2464 - val_mean_absolute_error: 0.3596 - val_nse: 0.3115
Epoch 94/200
1/1 [==============================] - ETA: 0s - loss: 0.2972 - mean_absolute_error: 0.3997 - nse: 0.7435
Epoch 00094: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.2972 - mean_absolute_error: 0.3997 - nse: 0.7435 - val_loss: 0.2478 - 

Epoch 116/200
1/1 [==============================] - ETA: 0s - loss: 0.2752 - mean_absolute_error: 0.4049 - nse: 0.6186
Epoch 00116: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.2752 - mean_absolute_error: 0.4049 - nse: 0.6186 - val_loss: 0.2315 - val_mean_absolute_error: 0.3488 - val_nse: 0.3550
Epoch 117/200
1/1 [==============================] - ETA: 0s - loss: 0.3283 - mean_absolute_error: 0.4075 - nse: 0.6887
Epoch 00117: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3283 - mean_absolute_error: 0.4075 - nse: 0.6887 - val_loss: 0.2367 - val_mean_absolute_error: 0.3490 - val_nse: 0.3385
Epoch 118/200
1/1 [==============================] - ETA: 0s - loss: 0.3472 - mean_absolute_error: 0.4334 - nse: 0.7101
Epoch 00118: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3472 - mean_absolute_error: 0.4334 - nse: 0.7101 - val_loss: 0.2396

Epoch 140/200
1/1 [==============================] - ETA: 0s - loss: 0.3482 - mean_absolute_error: 0.4312 - nse: 0.7136
Epoch 00140: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3482 - mean_absolute_error: 0.4312 - nse: 0.7136 - val_loss: 0.2613 - val_mean_absolute_error: 0.3644 - val_nse: 0.2644
Epoch 141/200
1/1 [==============================] - ETA: 0s - loss: 0.2746 - mean_absolute_error: 0.3876 - nse: 0.7578
Epoch 00141: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.2746 - mean_absolute_error: 0.3876 - nse: 0.7578 - val_loss: 0.2691 - val_mean_absolute_error: 0.3711 - val_nse: 0.2427
Epoch 142/200
1/1 [==============================] - ETA: 0s - loss: 0.3178 - mean_absolute_error: 0.4222 - nse: 0.6590
Epoch 00142: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3178 - mean_absolute_error: 0.4222 - nse: 0.6590 - val_loss: 0.2703

Epoch 164/200
1/1 [==============================] - ETA: 0s - loss: 0.3220 - mean_absolute_error: 0.4141 - nse: 0.6746
Epoch 00164: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3220 - mean_absolute_error: 0.4141 - nse: 0.6746 - val_loss: 0.2794 - val_mean_absolute_error: 0.3851 - val_nse: 0.2183
Epoch 165/200
1/1 [==============================] - ETA: 0s - loss: 0.3059 - mean_absolute_error: 0.4154 - nse: 0.6919
Epoch 00165: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.3059 - mean_absolute_error: 0.4154 - nse: 0.6919 - val_loss: 0.2906 - val_mean_absolute_error: 0.3894 - val_nse: 0.1872
Epoch 166/200
1/1 [==============================] - ETA: 0s - loss: 0.2924 - mean_absolute_error: 0.4073 - nse: 0.6984
Epoch 00166: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.2924 - mean_absolute_error: 0.4073 - nse: 0.6984 - val_loss: 0.3034

Epoch 188/200
1/1 [==============================] - ETA: 0s - loss: 0.2824 - mean_absolute_error: 0.4019 - nse: 0.7499
Epoch 00188: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.2824 - mean_absolute_error: 0.4019 - nse: 0.7499 - val_loss: 0.2621 - val_mean_absolute_error: 0.3703 - val_nse: 0.2681
Epoch 189/200
1/1 [==============================] - ETA: 0s - loss: 0.2971 - mean_absolute_error: 0.4193 - nse: 0.7362
Epoch 00189: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.2971 - mean_absolute_error: 0.4193 - nse: 0.7362 - val_loss: 0.2610 - val_mean_absolute_error: 0.3717 - val_nse: 0.2714
Epoch 190/200
1/1 [==============================] - ETA: 0s - loss: 0.2832 - mean_absolute_error: 0.4030 - nse: 0.7442
Epoch 00190: val_loss did not improve from 0.21422
1/1 [==============================] - 3s 3s/step - loss: 0.2832 - mean_absolute_error: 0.4030 - nse: 0.7442 - val_loss: 0.2584

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [27]:
#train_df.shape, val_df.shape, test_df.shape

In [28]:
#-----------------------------------------------------------------------------------------------------------------
#         *****************          **************************          ***************        ***************
#     ************************      ***************************      **********************     ****        ****
#    ********          *********    **************************      ******            ******    ****        ****
#    ********          *********            *********              ******              ******   ****        ****          
#    ********                               *********              ******              ******   *****      *****       
#     **************                        *********              ******              ******   ***************   
#       ***********************             *********              ******              ******   **************           
#                    ************           *********              ******              ******   *****                      
#                        ********           *********              ******              *****    *****                     
#     *******            ********           *********              ******              *****    *****                     
#    ******            *********            *********               ******            ******    *****                     
#    **************************             *********                *********************      *****                   
#       ********************                ********                   *****************        ******                    
#-----------------------------------------------------------------------------------------------------------------


In [29]:
from core.models import GRUModel, compile

In [30]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [37]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  11.893088
predict_day :  5
------------------------
1.8863925 5.7730412 2.4027154
nse3 :  [0.51699996]
pbias3 :  [14.848611]
save/yeong/models/do/
year : 2015 ~ 2020
run :  range(0, 1)
target :  do
train=8, test=1, val=1
---------------
[0.51699996]
[14.848611]


In [32]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

100/100 [==============================] - 9s 87ms/step - loss: 0.2166 - mean_absolute_error: 0.3532 - nse: 0.3954


[0.21658727526664734, 0.35319623351097107, 0.39539992809295654]

In [33]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [34]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: float() argument must be a string or a number, not 'NoneType'